In [1]:
clean_up = True
%run StdPackages.ipynb
d['data'] = os.path.join(d['data'], 'IO2018')

# GR18: Data

### 1. Load data

In [2]:
name = 'GR18'
error = 1e-7 # tolerance when testing IO balance
db = GpyDB(pickle_path = os.path.join(d['data'], f'IO_{name}'))
db.name = name+'_ModelData'
glob = CGE_globals.SmallOpen(kwargs_vals = {'t': range(2016,2051)}) # global settings used throughout; interest rates, long run growth rates, time index etc.

### 2. Production module

Specify a general mapping that we'll apply to (most) production sectors:

In [3]:
mFull = pd.MultiIndex.from_tuples([('KELBM', 'RxE'), ('KELBM', 'KELB'), ('KELB', 'iB'), ('KELB','KEL'), ('KEL','L'), ('KEL','KE'), ('KE','iM'), ('KE','35011_input')], names = ['n','nn'])

For the sectors that do not demand energy good (```35011```), remove the KE nest: 

In [4]:
m_noE = pd.MultiIndex.from_tuples([('KELBM', 'RxE'), ('KELBM', 'KELB'), ('KELB', 'iB'), ('KELB','KEL'), ('KEL','L'), ('KEL','iM')], names = ['n','nn'])
s_noE = adj.rc_pd(db.get('s_p'), ('not', db.get('vD').xs('35011')))
m = pyDatabases.cartesianProductIndex([s_noE, m_noE])

All sectors use both ```iB``` and ```iM``` - so, we finally check for whether all sectors deamnd labor:

In [5]:
m_noL = pd.MultiIndex.from_tuples([('KELBM', 'RxE'), ('KELBM', 'KELB'), ('KELB', 'iB'), ('KELB','KE'), ('KE','iM'), ('KE','35011_input')], names = ['n','nn'])
s_noL = adj.rc_pd(db.get('s_p'), ('not', db.get('vD').xs('L')))
m = m.union(pyDatabases.cartesianProductIndex([s_noL, m_noL]))

For all other sectors, use the full nesting structure:

In [6]:
mFull = pd.MultiIndex.from_tuples([('KELBM', 'RxE'), ('KELBM', 'KELB'), ('KELB', 'iB'), ('KELB','KEL'), ('KEL','L'), ('KEL','KE'), ('KE','iM'), ('KE','35011_input')], names = ['n','nn'])
sFull = adj.rc_pd(db.get('s_p'), ('not', ('or', [s_noE, s_noL])))
m = m.union(pyDatabases.cartesianProductIndex([sFull, mFull]))

Replace the upper-most level (KELBM) with the name of the sector:

In [7]:
df = m.to_frame(index=False)
df.loc[df.n == 'KELBM','n'] = df.loc[df.n == 'KELBM', 's']
m = pd.MultiIndex.from_frame(df)

For [s,n] in ```dImport[s,n,nn]```, we define a mapping from ```RxE```to an intermediate good ```RxEym_x``` that is again an aggregate of the domestic/foreign goods ```x,x_F```.

In [8]:
df = adj.rc_pd(db.get('dImport'), db.get('s_p')).to_frame(index=False).assign(m= lambda x: 'RxEym_'+x['n'], u = 'RxE', n = lambda x: x['n']+'_input')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','m']]).rename(['s','n','nn']))
m = m.union(pd.MultiIndex.from_frame(df[['s','m','n']]).rename(['s','n','nn']))
m = m.union(pd.MultiIndex.from_frame(df[['s','m','nn']]).rename(['s','n','nn']))

For [s,n] in ```dImport_dom[s,n]```, we define a mapping from ```RxE```to the domestic goods:

In [9]:
df = adj.rc_pd(db.get('dImport_dom'), db.get('s_p')).to_frame(index=False).assign(u = 'RxE', n = lambda x: x['n']+'_input')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn']))

For [s,n] in ```dImport_for[s,n]```, we define a mapping from ```RxE```to the forein goods:

In [10]:
df = adj.rc_pd(db.get('dImport_for'), db.get('s_p')).to_frame(index=False).assign(u = 'RxE')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn']))

In [11]:
db['nestProduction'] = m

### 3. Investment module

For the investment module, the nesting structure is similar to the materials nest in the production module:

### 8. Quantities

Broadcast all variables using the time index (repeat variables for all $t$):

In [17]:
def addT(symbol, t):
    return adjMultiIndex.bc(symbol, t).reorder_levels(['t']+symbol.index.names if 't' not in symbol.index.names else symbol.index.names)
[db.__setitem__(k, addT(db.get(k), glob.db['t'].vals)) for k in db.getTypes(['variable','scalar_variable'])];

## Export:

In [19]:
aggregateDB.readSets(db) # read sets from the symbols in data

In [20]:
db.export(repo = d['data'])
with open(f"{d['data']}\\glob_{name}", "wb") as file:
    pickle.dump(glob,file)